In [50]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


In [52]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [53]:
examples = [
    {
        "movie": "Inception",
        "answer": """
        Here is what I know:
        Director: Christopher Nolan
        Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy
        Budget: $160 million
        Revenue: $828 million
        Genre: Sci-Fi, Action, Thriller
        Synopsis: A skilled thief who steals information by infiltrating the subconscious is offered a chance to have his criminal history erased if he can successfully perform inception — planting an idea into a target's subconscious.
        """,
    },
    {
        "movie": "Parasite",
        "answer": """
        I know this:
        Director: Bong Joon-ho
        Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Park So-dam, Choi Woo-shik
        Budget: $11 million
        Revenue: $260 million
        Genre: Drama, Thriller, Dark Comedy
        Synopsis: A poor family schemes to become employed by a wealthy household by posing as unrelated, highly qualified individuals, leading to unexpected consequences.
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        I know this:
        Director: Francis Ford Coppola
        Cast: Marlon Brando, Al Pacino, James Caan, Robert Duvall
        Budget: $6 million
        Revenue: $250 million
        Genre: Crime, Drama
        Synopsis: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
        """,
    },
    {
        "movie": "Interstellar",
        "answer": """
        Here is what I know:
        Director: Christopher Nolan
        Cast: Matthew McConaughey, Anne Hathaway, Jessica Chastain, Michael Caine
        Budget: $165 million
        Revenue: $677 million
        Genre: Sci-Fi, Drama, Adventure
        Synopsis: In a future where Earth is becoming uninhabitable, a team of astronauts travels through a wormhole in search of a new home for humanity.
        """,
    },
    {
        "movie": "La La Land",
        "answer": """
        I know this:
        Director: Damien Chazelle
        Cast: Ryan Gosling, Emma Stone
        Budget: $30 million
        Revenue: $450 million
        Genre: Musical, Romance, Drama
        Synopsis: A jazz musician and an aspiring actress fall in love while pursuing their dreams in Los Angeles, facing choices between love and career.
        """,
    },
]


In [54]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

In [55]:
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [56]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a film critic, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

In [57]:
chain = final_prompt | chat

chain.invoke({"movie": "La La Land"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "movie": "La La Land"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "movie": "La La Land"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a film critic, you give short answers.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
   

AIMessageChunk(content='I know this:\n        Director: Damien Chazelle\n        Cast: Ryan Gosling, Emma Stone\n        Budget: $30 million\n        Revenue: $450 million\n        Genre: Musical, Romance, Drama\n        Synopsis: A jazz musician and an aspiring actress fall in love while pursuing their dreams in Los Angeles, facing choices between love and career.')